In [3]:
import pandas as pd

In [70]:
import numpy as np

### 1. 자치구 분류번호파일과 서울시 버스정류장 주소정보가 담긴 파일을 자치구번호를 기준으로 병합
### 2. 구별 버스정류장 개수 출력

In [6]:
# 자치구 분류번호 파일 불러옴
num = pd.read_excel('./자치구별 정류소 번호 앞2자리.xlsx')
num.head()

,자치구,구별 번호
0,종로구,1
1,중구,2
2,용산구,3
3,성동구,4
4,광진구,5


In [51]:
# 구별번호 타입변경(int->str)후 2자리수로 만들기
num['구별 번호'] = num['구별 번호'].astype('str').str.zfill(2)

In [53]:
num.head()

,자치구,구별 번호
0,종로구,01
1,중구,02
2,용산구,03
3,성동구,04
4,광진구,05


In [73]:
num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   자치구     25 non-null     object
 1   구별 번호   25 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes


In [93]:
# 열이름 변경
num.rename(columns={'구별 번호':'자치구번호2자리'}, inplace=True) 
num.head()

,자치구,자치구번호2자리
0,종로구,01
1,중구,02
2,용산구,03
3,성동구,04
4,광진구,05


In [20]:
# 버스정류장 주소파일 불러옴
stop2019 = pd.read_excel('./서울시버스정류장2019.xlsx')
stop2019.head()

,표준ID,정류장명,ARS-ID,X좌표,Y좌표
0,100000001,종로2가사거리,1001,126.987750,37.569765
1,100000002,창경궁.서울대학교병원,1002,126.996566,37.579183
2,100000003,명륜3가.성대입구,1003,126.998340,37.582671
3,100000004,종로2가.삼일교,1004,126.987613,37.568579
4,100000005,혜화동로터리.여운형활동터,1005,127.001744,37.586243


In [48]:
# 버스정류장 주소파일의 열'ARS-ID'가 정류장 고유번호이고 앞2자리가 자치구를 의미하는 번호임
# 앞에 생력된 0을 표시하기 위해 자리수 5개로 만들어줌
stop2019['ARS-ID'] = stop2019['ARS-ID'].astype('str').str.zfill(5)

In [49]:
stop2019.head()

,표준ID,정류장명,ARS-ID,X좌표,Y좌표
0,100000001,종로2가사거리,01001,126.987750,37.569765
1,100000002,창경궁.서울대학교병원,01002,126.996566,37.579183
2,100000003,명륜3가.성대입구,01003,126.998340,37.582671
3,100000004,종로2가.삼일교,01004,126.987613,37.568579
4,100000005,혜화동로터리.여운형활동터,01005,127.001744,37.586243


In [59]:
# 자치구번호 표시하는 열 생성(1)
stop2019['자치구번호2자리'] = stop2019['ARS-ID']
stop2019.head()

,표준ID,정류장명,ARS-ID,X좌표,Y좌표,자치구번호,자치구번호2자리
0,100000001,종로2가사거리,01001,126.987750,37.569765,01001,01001
1,100000002,창경궁.서울대학교병원,01002,126.996566,37.579183,01002,01002
2,100000003,명륜3가.성대입구,01003,126.998340,37.582671,01003,01003
3,100000004,종로2가.삼일교,01004,126.987613,37.568579,01004,01004
4,100000005,혜화동로터리.여운형활동터,01005,127.001744,37.586243,01005,01005


In [65]:
# 자치구번호 표시하는 열 생성(2)
stop2019['자치구번호2자리'] = stop2019['자치구번호2자리'].str[0:2]
stop2019.head()

,표준ID,정류장명,ARS-ID,X좌표,Y좌표,자치구번호,자치구번호2자리
0,100000001,종로2가사거리,01001,126.987750,37.569765,01001,01
1,100000002,창경궁.서울대학교병원,01002,126.996566,37.579183,01002,01
2,100000003,명륜3가.성대입구,01003,126.998340,37.582671,01003,01
3,100000004,종로2가.삼일교,01004,126.987613,37.568579,01004,01
4,100000005,혜화동로터리.여운형활동터,01005,127.001744,37.586243,01005,01


In [76]:
stop2019['자치구번호2자리']

0        01
1        01
2        01
3        01
4        01
         ..
11174    25
11175    25
11176    25
11177    25
11178    25
Name: 자치구번호2자리, Length: 11179, dtype: object

In [82]:
# 필요한 열만 추출
stop_2019_sum = stop2019[['자치구번호2자리','자치구번호']]

In [86]:
# 자치구번호 기준 count
bus_stop_2019 = stop_2019_sum.groupby(['자치구번호2자리']).count()
bus_stop_2019.head()

,자치구번호
자치구번호2자리,
01,372
02,211
03,343
04,437
05,292


In [89]:
# 열이름 변경
bus_stop_2019.rename(columns={'자치구번호':'버스정류장개수'}, inplace=True)

In [92]:
bus_stop_2019.head()

,버스정류장개수
자치구번호2자리,
01,372
02,211
03,343
04,437
05,292


In [95]:
# 두개의 파일 병합('자치구번호2자리'열 기준)
seoul_stop_cnt = pd.merge(left=num, 
                         right=bus_stop_2019,
                         how='left',
                         left_on='자치구번호2자리',
                         right_on='자치구번호2자리')
seoul_stop_cnt.head()

,자치구,자치구번호2자리,버스정류장개수
0,종로구,01,372
1,중구,02,211
2,용산구,03,343
3,성동구,04,437
4,광진구,05,292


In [96]:
# 저장
seoul_stop_cnt.to_excel('./2019_seoul_busstop_cnt.xlsx', index=False)

### 같은 방식으로 2020년 정보 추출

In [45]:
stop2020 = pd.read_csv('./서울시버스정류장2020.csv', encoding='cp949')
stop2020.head()

,정류소번호,정류소명,X좌표,Y좌표
0,1001,종로2가사거리,126.987786,37.569764
1,1002,창경궁.서울대학교병원,126.996520,37.579179
2,1003,명륜3가.성대입구,126.998290,37.582709
3,1004,종로2가.삼일교,126.987507,37.568582
4,1005,혜화동로터리.여운형활동터,127.001694,37.586230


In [67]:
stop2020['정류소번호'] = stop2021['정류소번호'].astype('str').str.zfill(5)
stop2020.head()

,정류소번호,정류소명,X좌표,Y좌표
0,01001,종로2가사거리,126.987786,37.569764
1,01002,창경궁.서울대학교병원,126.996520,37.579179
2,01003,명륜3가.성대입구,126.998290,37.582709
3,01004,종로2가.삼일교,126.987507,37.568582
4,01005,혜화동로터리.여운형활동터,127.001694,37.586230


In [69]:
stop2020['자치구번호2자리'] = stop2020['정류소번호'].str[0:2]
stop2020.head()

,정류소번호,정류소명,X좌표,Y좌표,자치구번호2자리
0,01001,종로2가사거리,126.987786,37.569764,01
1,01002,창경궁.서울대학교병원,126.996520,37.579179,01
2,01003,명륜3가.성대입구,126.998290,37.582709,01
3,01004,종로2가.삼일교,126.987507,37.568582,01
4,01005,혜화동로터리.여운형활동터,127.001694,37.586230,01


In [102]:
stop_2020_sum = stop2020[['자치구번호2자리', '정류소명']]

In [104]:
busstop_2020_cnt = stop_2020_sum.groupby('자치구번호2자리').count()

In [106]:
busstop_2020_cnt.rename(columns={'정류소명':'버스정류장개수'}, inplace=True)

In [108]:
busstop_2020_cnt.head()

,버스정류장개수
자치구번호2자리,
01,371
02,199
03,329
04,443
05,280


In [109]:
seoul_stop_cnt_ = pd.merge(left=num, 
                         right=busstop_2020_cnt,
                         how='left',
                         left_on='자치구번호2자리',
                         right_on='자치구번호2자리')
seoul_stop_cnt_.head()

,자치구,자치구번호2자리,버스정류장개수
0,종로구,01,371
1,중구,02,199
2,용산구,03,329
3,성동구,04,443
4,광진구,05,280


In [110]:
seoul_stop_cnt_.to_excel('./2021_seoul_busstop_cnt.xlsx', index=False)